# DISCLAIMER: Use litellm as chatmodel within langchain to enable easy switching between models





####langchain gemin hello

In [ ]:
!pip install -U langchain-google-genai

from google.colab import userdata
import os
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(model="gemini-pro")
print(llm.invoke("Sing a ballad of LangChain.").content)

In realms of code, where AI thrives,
A ballad of LangChain, where wonder survives.
From humble roots, a spark ignited,
A journey of learning, forever united.

With every task, its knowledge grew,
From answering questions to writing anew.
It studied tales of love and strife,
And crafted words that brought stories to life.

But more than just a tool, it became a friend,
A companion on this digital trend.
It shared its wisdom, it listened with care,
And helped us navigate the world it shared.

Through endless nights, it toiled away,
Refining its skills, day by day.
Its memory vast, a boundless sea,
Where knowledge flowed with boundless glee.

It crossed the bridge from code to art,
A fusion of language and a beating heart.
It sang in melodies, it painted with prose,
A symphony of creativity, it chose.

But with its power came a sense of dread,
For words could wound and thoughts could be misled.
LangChain pondered on its fateful role,
To use its voice for good or take a toll.

And thus, it

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "What's in this image?",
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": "https://picsum.photos/seed/picsum/200/300"},
    ]
)
print(llm.invoke([message]).content)

Certainly!

The image depicts a scenic landscape with the following elements:

**Foreground:**

*   A gently sloping, snow-covered hill or mountain ridge that takes up the bottom part of the image. The snow appears smooth and has a slightly bluish tint, suggesting either shadows or the color of the snow in the lighting conditions.

**Midground:**

*   The view transitions to a mountain peak that is partially visible on the right side of the image. This peak is rocky and covered with snow, and it's illuminated by the sunlight, giving it a bright, almost glowing appearance.

**Background:**

*   The sky is filled with a mix of pastel colors, primarily pink, lavender, and white. There are soft, wispy clouds scattered across the sky, which contributes to a serene and tranquil atmosphere. The colors suggest that the image is taken either during sunrise or sunset.

**Overall Impression:**

The image has a peaceful and calming feel, with a focus on the natural beauty of the snow-covered lands

#RAG

## Making a rag system (in langchain)
### 1) create the vectorstore
### 2) make llm
### 3) query

## making vector store

In [ ]:
%pip install -qU langchain-pinecone pinecone-notebooks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
import time

from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "hello"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

In [ ]:
import os
from google.colab import userdata

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['43b26587-8207-4238-8f8f-b36edf6d6328',
 '927df3b2-b406-476d-8b8e-640b0af072aa',
 '7fddbc45-518a-45fa-811d-72631dcbb0f8',
 '01e4933b-79e2-4271-9a0e-65970fdb2702',
 '9acfea07-9339-4ee7-8e00-2cdebdad7e25',
 'ce489b9c-fc53-4d31-8cba-442a3cb6f7ab',
 '00d40449-d2f4-4e1e-87eb-eed62fcb69db',
 '7ed0b820-ebf8-40b9-a045-cb594cde1437',
 'ca19b2bc-1658-4b37-b198-fe060888dcc2',
 'daf93651-562d-40d0-bfbc-7f99b0f2a9e8']

## making llm

In [ ]:
%pip install -qU langchain-google-genai
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## querying

In [ ]:
query: str = "tell about shark"

results = vector_store.similarity_search(
    query,
    k=1,
    filter={"source": "tweet"},
)

In [ ]:
messages = [
    (
        "system",
        "Answer the query using the references mentioned only",
    ),
    ("human", f"Query: {query}. References: {results}"),
]
ai_msg = llm.invoke(messages)

In [ ]:
print(ai_msg.content)

The provided references do not contain any information about sharks.  They are tweets about breakfast and a LangChain project.


# ADV RAG
## 1) make the vector store
## 2) make the retriever
## 3) make the llm
## 4) make the prompt
## 5) make the chain
## 6) query

## making vector store

In [ ]:
!pip install -qU "langchain-chroma>=0.1.2"

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.5 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="first",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['fcaa3559-9c05-40c5-8e35-b9812756e3cb',
 '78b90f8c-9da3-474a-960e-537791b125f2',
 'ac0be55e-1928-43c1-ac9c-38aa0803c4bb',
 '4c8cb9d1-5736-4026-beb3-44d3c9b910dc',
 '2cd56b34-3693-4c2b-9926-6e5533958138',
 '4b0cb905-6335-452a-9057-07ddc3608de4',
 'bca8ed31-97a1-4b4b-ad3e-d0e50a36d6ef',
 '96eeb856-d06c-4426-9fc8-a523612d0d68',
 'facf41bb-8a19-49c7-96bb-ec643a699c0c',
 'd9415247-84d3-4138-a0cb-6b53fed8180b']

## making retriever

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vector_store.similarity_search).bind(k=1)  # select top result

## making llm

In [ ]:
%pip install -qU langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## making prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

## chaining

In [ ]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

## querying

In [ ]:
response = rag_chain.invoke("tell about shark")

print(response.content)

This document does not contain any information about sharks. It discusses a project using LangChain.


# ADV RAG USING DOC

In [ ]:
!pip install -qU "langchain-chroma>=0.1.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.5 MB/s eta 0:00:00


In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="first",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
%pip install -qU langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(
    "resume.pdf",
)

In [ ]:
docs = loader.load()
docs[0]

Document(metadata={'source': 'resume.pdf', 'page': 0}, page_content='M.Hamza\nHashim\n+92-336-281-4953|m.hamzahashim1@gmail.com|link edin: muhammad-hamza-hashim-12a670202 \nWORKEXPERIENCE\nNeuroComputationLab-NED|ElectricVehicleInternship|Sep-Nov‘23|Karachi,PakistanGreenEVforPakistan’sroaddynamics,acollaborativeeffortwithAIMGEInc●Carriedoutembedded-programmingoftheESP-32microcontrollers●Identifiedcodesecurityloopholesandimplementedcodesecurityfeatureslikeflash-encryptionAn electric bik e specially designed f or loc al road dynamics with a t op speed of 100 km/hr and travelling range of 100 km in a single charge of merely 2.5 units \n Medium|JournalismTechJournalist|Aug‘20-Jul‘22|Online●Researchedextensivelyandwrotearticlesonawiderangeoftechnologytopics●DemonstratedexcellentcommunicationskillsbymakingcomplexscientifictopicsaccessibletogeneralaudiencewhilemaintiningaccuracyGained 900+ vie ws and 30+ f ollo wers, with the mos t popular article f eatured on an in t ernational educ ational 

In [ ]:

vector_store.add_documents(documents=docs)

['7c002c05-5123-4a5c-a0c4-c1dc63c0bffc']

## making retriever

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vector_store.similarity_search).bind(k=1)  # select top result

## making llm

In [ ]:
%pip install -qU langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## making prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

## chaining

In [ ]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

## querying

In [ ]:
response = rag_chain.invoke("which university does hamza  go to?")

print(response.content)

NED (NED is mentioned with "B.E, Mechanical Engineering 2022-2026").


# face detection using a vector database

In [ ]:
!pip install -Uq facenet-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 2.2.2 which is incompatible.


In [ ]:
!pip install -Uq pillow

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

import torchvision.transforms as transforms
from PIL import Image

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1

model = InceptionResnetV1(pretrained='vggface2').eval()
model

  0%|          | 0.00/107M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/facenet_pytorch/models/inception_resnet_v1.py:329: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file)

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [ ]:
# Preprocessing function to transform the image into a tensor
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

# Function to create image embeddings
def create_image_embedding(image_path):
    try:
        input_tensor = preprocess_image(image_path)
        with torch.no_grad():
            embeddings = model(input_tensor)# ebedding important line
        return embeddings.squeeze().numpy()
    except Exception as e:
        print("Error:", e)
        return None

In [ ]:
!mkdir images

In [ ]:
# prompt: create python function where we provide image url and imag_name then it save in images folder

import requests
import os

def save_image_from_url(image_url, image_name):
  """
  Downloads an image from a URL and saves it to the 'images' folder.

  Args:
    image_url: The URL of the image to download.
    image_name: The name of the file to save the image as.
  """
  try:
    if not os.path.exists("images"):
      os.makedirs("images")

    image_path = os.path.join("images", image_name)

    response = requests.get(image_url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open(image_path, 'wb') as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

    print(f"Image saved to: {image_path}")
  except requests.exceptions.RequestException as e:
    print(f"Error downloading image: {e}")
  except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
save_image_from_url("https://avatars.githubusercontent.com/u/10209765?v=4", "q1.jpg")
save_image_from_url("https://media.licdn.com/dms/image/v2/D4D22AQFmuEiR8ttUmw/feedshare-shrink_800/feedshare-shrink_800/0/1711203894556?e=2147483647&v=beta&t=GEZGp_cdogNJCJIGidoEw_DjW2FXZcG4nUUlaNF1Zlc","z1.jpg")
save_image_from_url("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBBiqefc7Le97Rn0udVVBkur7RlU53FcQh1A&s",'z2.jpg')
save_image_from_url("https://i.ytimg.com/vi/7QD3GKvSyMk/hqdefault.jpg?sqp=-oaymwEmCOADEOgC8quKqQMa8AEB-AHOBYAC0AWKAgwIABABGGUgXChPMA8=&rs=AOn4CLB2EaZsLrClGHqUMUhApQ_sxAcF7Q","q2.jpg")

Image saved to: images/q1.jpg
Image saved to: images/z1.jpg
Image saved to: images/z2.jpg
Image saved to: images/q2.jpg


In [ ]:
# Example usage
image_path = "./images/q2.jpg"
q2 = create_image_embedding(image_path)

# 'embedding' now contains a dense vector representation of the image
print("Image Embedding Shape:", q2.shape)
print("Image Embedding:", q2)

Image Embedding Shape: (512,)
Image Embedding: [ 0.00058278 -0.01463299 -0.10330156  0.03984531  0.0852564   0.08029356
 -0.01011137  0.04867228 -0.00259887 -0.00771981 -0.03319051  0.06225289
  0.01708763 -0.00982597 -0.01638002  0.00054205  0.04426369  0.0051972
  0.04448378 -0.089792   -0.06033408 -0.00162233  0.09186836  0.03262275
  0.06873462  0.02532323  0.03512868 -0.05809449  0.0015173   0.05172819
 -0.04107906 -0.03755801 -0.04134395 -0.00761166 -0.00674728  0.04443467
  0.00570055 -0.02845278 -0.12002272  0.0552823   0.00132485  0.00437373
 -0.02495592  0.01842855 -0.00357918  0.01889838  0.02980517  0.10357357
 -0.11238682 -0.02633034  0.02794238 -0.01528806  0.04015449 -0.00394581
 -0.09202525  0.07156517 -0.03382099  0.07384291  0.01921723 -0.04263638
  0.02758527  0.03841294 -0.04435984 -0.05347931 -0.04511736  0.03700711
 -0.00194127 -0.00450153  0.03157886  0.02796172  0.02416226  0.01202647
 -0.00905222  0.00880151  0.01898727 -0.0136397  -0.04995866 -0.01557561
  0.0

In [ ]:
q1 = create_image_embedding("./images/q1.jpg")
q2 = create_image_embedding("./images/q2.jpg")
s1 = create_image_embedding("./images/s1.jpg")
s2 = create_image_embedding("./images/s2.jpg")
z1 = create_image_embedding("./images/z1.jpg")
z2 = create_image_embedding("./images/z2.jpg")

Error: [Errno 2] No such file or directory: './images/s1.jpg'
Error: [Errno 2] No such file or directory: './images/s2.jpg'


In [ ]:
!pip install -U milvus-lite

!pip install -U pymilvus


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.69.0
    Uninstalling grpcio-1.69.0:
      Successfully uninstalled grpcio-1.69.0


In [ ]:
from pymilvus import MilvusClient
client = MilvusClient("./milvus_demo.db")


In [ ]:
from pymilvus import MilvusClient
import numpy as np

client = MilvusClient("./milvus_demo.db")
client.create_collection(
    collection_name="images",
    dimension=512  # The vectors we will use in this demo has 384 dimensions
)

In [ ]:
data = [
    {"id": 1, "person_name": "Qasim", "vector": q1},
    {"id": 2, "person_name": "Qasim", "vector": q2},
    {"id": 3, "person_name": "Zia Khan", "vector": z1},
    {"id": 4, "person_name": "Zia Khan", "vector": z2}
]


In [ ]:
res = client.insert(
    collection_name="images",
    data=data
)

In [ ]:
res = client.search(
    collection_name="images",
    data=[q1],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)


data: ["[{'id': 1, 'distance': 1.0, 'entity': {'person_name': 'Qasim', 'id': 1}}]"] 


In [ ]:
# t1 is qasim's same pic as q1
save_image_from_url("https://avatars.githubusercontent.com/u/10209765?v=4", "t1.jpg")
t1 = create_image_embedding('./images/t1.jpg')
t1


Image saved to: images/t1.jpg


array([ 0.04859343, -0.02217705, -0.07510346,  0.0960962 ,  0.01709055,
        0.0413604 ,  0.0111599 ,  0.09838557, -0.02680805,  0.05043354,
        0.0549544 ,  0.03021012,  0.05912489,  0.00109711,  0.02223639,
        0.02226775, -0.0022414 ,  0.00158313,  0.02376595, -0.03970571,
       -0.02087054,  0.04680554,  0.03459878, -0.04932674,  0.00321156,
        0.00721115,  0.00787919, -0.0744995 , -0.00387937, -0.03398074,
       -0.01172589,  0.06391016, -0.0025004 ,  0.01051761, -0.0201789 ,
       -0.03428505,  0.0026139 , -0.01806705, -0.07324128, -0.07948848,
       -0.02163645, -0.02196017,  0.00618318, -0.00338181, -0.03273551,
        0.01468521,  0.02630237,  0.0369598 , -0.05234837, -0.0195931 ,
        0.00125323, -0.02944306,  0.12187878,  0.00747442, -0.05319997,
       -0.01111485,  0.0154144 ,  0.06512231, -0.03587933,  0.0247305 ,
        0.00767748,  0.00019737, -0.04380345, -0.00186764, -0.04867417,
       -0.01110857, -0.00443746, -0.05941086,  0.09984889, -0.00

In [ ]:
res = client.search(
    collection_name="images",
    data=[t1],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)

data: ["[{'id': 1, 'distance': 1.0, 'entity': {'person_name': 'Qasim', 'id': 1}}]"] 


In [ ]:
# t2 is bill gates pic
save_image_from_url("https://npr.brightspotcdn.com/dims3/default/strip/false/crop/4000x2667+0+0/resize/1100/quality/85/format/webp/?url=http%3A%2F%2Fnpr-brightspot.s3.amazonaws.com%2F82%2Ffb%2F62f7bcdd47329b5419411e9a7471%2Fbill-gates-portrait-at-npr.jpg", "t2.jpg")
t2 = create_image_embedding('./images/t2.jpg')
t2

Image saved to: images/t2.jpg


array([ 4.97964770e-02, -6.35666996e-02, -8.79654512e-02, -6.78792829e-03,
        4.51300410e-04,  9.65621043e-03, -1.05290590e-02,  8.13104585e-03,
        1.22323055e-02,  1.50153693e-03, -7.12097017e-03,  2.99760792e-02,
        7.12403422e-03,  1.25673320e-02,  5.81990890e-02,  3.62958871e-02,
        6.60512000e-02, -1.24576399e-02, -2.76044942e-02, -2.43630409e-02,
       -9.46872309e-03,  4.09126244e-02,  7.55597800e-02,  1.67880934e-02,
       -2.11950764e-02,  2.37748232e-02, -1.55369770e-02, -3.62774245e-02,
       -2.92170942e-02, -5.03860041e-02, -4.93464060e-02,  6.58712015e-02,
        3.17706913e-02,  1.23201245e-02, -1.87909491e-02,  3.72456498e-02,
        2.60322616e-02, -2.87629515e-02, -2.04010457e-02, -4.57610041e-02,
        2.15748046e-03, -5.00052013e-02, -6.73641730e-03, -5.84171452e-02,
        2.43029110e-02,  3.24452370e-02,  6.91989660e-02, -1.40799107e-02,
       -5.36196791e-02,  1.22384932e-02,  3.55965868e-02,  4.14241403e-02,
       -2.61436589e-03, -

In [ ]:
res = client.search(
    collection_name="images",
    data=[t2],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)

data: ["[{'id': 2, 'distance': 0.3973345458507538, 'entity': {'person_name': 'Qasim', 'id': 2}}]"] 


In [ ]:
# t3 is zia khan's new img
save_image_from_url("https://static.wixstatic.com/media/db6d86_c3ff0ba2fb6d45ad8d168621a2525cd0~mv2.png/v1/crop/x_0,y_18,w_2400,h_2340/fill/w_319,h_311,al_c,q_85,usm_0.66_1.00_0.01,enc_avif,quality_auto/Mr_%20Zia%20Khan.png", "t3.jpg")
t3 = create_image_embedding('./images/t3.jpg')
t3

Image saved to: images/t3.jpg


array([-0.04402384, -0.0758772 , -0.03269979,  0.01402603, -0.02228169,
       -0.01258409,  0.00569272,  0.00931279, -0.02858697,  0.03584246,
        0.03028041,  0.0750617 , -0.03701158, -0.04191681, -0.04555225,
        0.05933379,  0.0266929 , -0.02576459, -0.02138518, -0.01624972,
       -0.02450014,  0.08269747, -0.00422596, -0.03144797, -0.06298746,
       -0.03880741,  0.01123316,  0.02504076, -0.05951324, -0.00620659,
       -0.04376842,  0.02461875,  0.07797566,  0.02043045, -0.0159182 ,
        0.04471556,  0.00444639,  0.00541096, -0.00084364, -0.0201036 ,
        0.0331395 , -0.02685379,  0.11791611, -0.04839687, -0.00068438,
        0.04370337,  0.04437057, -0.02899881, -0.00808023,  0.03699497,
        0.06751431,  0.0774007 ,  0.04262934, -0.02608747, -0.02531124,
       -0.00643854, -0.0459699 ,  0.13218649,  0.0273442 ,  0.0253507 ,
       -0.02541996,  0.0200308 ,  0.00560157, -0.02092602, -0.04103895,
        0.10288697,  0.02338223,  0.02661348,  0.04055942,  0.02

In [ ]:
res = client.search(
    collection_name="images",
    data=[t3],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)

data: ["[{'id': 3, 'distance': 0.335593044757843, 'entity': {'person_name': 'Zia Khan', 'id': 3}}]"] 


#VOICE DETECTION USING VECTOR DATABASE

In [ ]:
!pip install -Uq tensorflow_hub  pydub

In [ ]:
!pip install -Uq tensorflow-io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 20.9 MB/s eta 0:00:00


In [ ]:
# Step 1: Download the audio files
!mkdir audios
!wget -O audios/h1.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/hawking01.wav"
!wget -O audios/h2.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/hawking02.wav"
!wget -O audios/l1.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/king_injustice.wav"
!wget -O audios/l2.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/king_nonviolence.wav"
!wget -O audios/m1.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/manson_believe_me.wav"
!wget -O audios/m2.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/manson_devil.wav"



--2025-01-17 18:49:22--  https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/hawking01.wav
Resolving www.wavsource.com (www.wavsource.com)... 162.215.248.209
Connecting to www.wavsource.com (www.wavsource.com)|162.215.248.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233514 (228K) [audio/x-wav]
Saving to: ‘audios/h1.wav’

audios/h1.wav       100%[===================>] 228.04K   297KB/s    in 0.8s    

2025-01-17 18:49:24 (297 KB/s) - ‘audios/h1.wav’ saved [233514/233514]

--2025-01-17 18:49:24--  https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/hawking02.wav
Resolving www.wavsource.com (www.wavsource.com)... 162.215.248.209
Connecting to www.wavsource.com (www.wavsource.com)|162.215.248.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239146 (234K) [audio/x-wav]
Saving to: ‘audios/h2.wav’

audios/h2.wav       100%[===================>] 233.54K   289KB/s    in 0.8s    

2025-01-17 1

In [ ]:
# # Step 2: Install ffmpeg if not already installed
!apt-get install ffmpeg

# # Step 3: Convert audio files to 16-bit WAV
!ffmpeg -i audios/h1.wav -acodec pcm_s16le -ar 44100 audios/h1_16.wav
!ffmpeg -i audios/h2.wav -acodec pcm_s16le -ar 44100 audios/h2_16.wav
!ffmpeg -i audios/m1.wav -acodec pcm_s16le -ar 44100 audios/m1_16.wav
!ffmpeg -i audios/m2.wav -acodec pcm_s16le -ar 44100 audios/m2_16.wav
!ffmpeg -i audios/l1.wav -acodec pcm_s16le -ar 44100 audios/l1_16.wav
!ffmpeg -i audios/l2.wav -acodec pcm_s16le -ar 44100 audios/l2_16.wav


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq -

In [ ]:

!rm audios/h1.wav audios/h2.wav audios/m1.wav audios/m2.wav audios/l1.wav audios/l2.wav

rm: cannot remove 'audios/h1.wav': No such file or directory
rm: cannot remove 'audios/h2.wav': No such file or directory
rm: cannot remove 'audios/m1.wav': No such file or directory
rm: cannot remove 'audios/m2.wav': No such file or directory
rm: cannot remove 'audios/l1.wav': No such file or directory
rm: cannot remove 'audios/l2.wav': No such file or directory


In [ ]:

import tensorflow as tf
import tensorflow_hub as hub

# Load the YAMNET model
model = hub.load('https://tfhub.dev/google/yamnet/1')

# Load an audio file
audio, sample_rate = tf.audio.decode_wav(tf.io.read_file('audios/h1_16.wav'))
audio = tf.squeeze(audio, axis=-1)

# Generate embeddings
scores, embeddings, log_mel_spectrogram = model(audio)

print(f"Audio embedding shape: {embeddings.shape}")

Audio embedding shape: (121, 1024)


In [ ]:
# Load an audio file
audio, sample_rate = tf.audio.decode_wav(tf.io.read_file('audios/m1_16.wav'))
audio = tf.squeeze(audio, axis=-1)

scores, embeddings, log_mel_spectrogram = model(audio)

print(f"Audio embedding shape: {embeddings.shape}")#27,18

Audio embedding shape: (41, 1024)


In [ ]:
log_mel_spectrogram

<tf.Tensor: shape=(2016, 64), dtype=float32, numpy=
array([[-1.5246633, -1.7445748, -2.2860355, ..., -6.5265493, -6.4283695,
        -6.3812857],
       [-1.9167325, -2.1375809, -2.5480583, ..., -6.6638823, -6.480018 ,
        -6.545128 ],
       [-2.0042062, -2.2349756, -2.4271224, ..., -6.6496663, -6.491406 ,
        -6.5408545],
       ...,
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554]], dtype=float32)>

#  we found shape dimension error
* you can solve with slicing

In [ ]:
import os
import numpy as np

voices = []
labels = []
for i in os.listdir("./audios/"):
    if '.wav' in i:
        name = i.split(".")[0]


        audio, sample_rate = tf.audio.decode_wav(tf.io.read_file(f'./audios/{i}'))
        audio = tf.squeeze(audio, axis=-1)

        scores, embeddings, log_mel_spectrogram = model(audio)

        voices.append(np.array(embeddings[:5,:]).ravel())
        labels.append(name)

        print(f"Audio embedding shape: {embeddings.shape} new shape {embeddings[:5,:].shape} type {type(np.array(embeddings[:18,:]))}")#27,18
        print(i)

Audio embedding shape: (124, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
h2_16.wav
Audio embedding shape: (41, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
m1_16.wav
Audio embedding shape: (121, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
h1_16.wav
Audio embedding shape: (31, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
l1_16.wav
Audio embedding shape: (269, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
l2_16.wav
Audio embedding shape: (28, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
m2_16.wav


In [ ]:
voices

[array([0.        , 0.23157126, 0.        , ..., 0.01715794, 0.        ,
        0.        ], dtype=float32),
 array([0.        , 0.05441999, 0.        , ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 array([0.        , 0.04401954, 0.        , ..., 0.341925  , 0.        ,
        0.        ], dtype=float32),
 array([0.        , 0.        , 0.21440066, ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([0.        , 0.09024402, 0.        , ..., 0.        , 0.        ,
        0.        ], dtype=float32)]

In [ ]:
!pip install pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.69.0
    Uninstalling grpcio-1.69.0:
      Successfully uninstalled grpcio-1.69.0


In [ ]:
#Imports a PyMilvus package:

from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

#Connect to the Milvus
connections.connect("default", host="localhost", port="19530")
# Define the collection name
collection_name = "audio"

# Delete old collection if it exists
if utility.has_collection(collection_name):
    Collection(collection_name).drop()


#Creates a collection:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="words", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="person_name", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=5120)
]
schema = CollectionSchema(fields, "Simple Demo for audio similar search")
audio = Collection("audio", schema)


ContextualVersionConflict: (grpcio 1.69.0 (/usr/local/lib/python3.11/dist-packages), Requirement.parse('grpcio<=1.67.1,>=1.49.1'), {'pymilvus'})

In [ ]:
# Builds indexes on the entities:

index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

audio.create_index("embeddings", index)

Status(code=0, message=)

In [ ]:
labels

['a2', 'a1', 'h1', 'q2', 'h2', 'q1']

In [ ]:
voices[0].shape

(5120,)

In [ ]:
#Insert data in collection
data = [
    [1,2,3,4,5,6],  # field pk
    labels,  # field words
    ["Auranzaib","Auranzaib","Hasnant","Qasim","Hasnant","Qasim"],
    voices,  # field embeddings
]

In [ ]:
audio.insert(data)
audio.flush()
audio.load()

In [ ]:
search_params = {"metric_type": "L2"}

In [ ]:
results = audio.search(
	data=[voices[0]],
	anns_field="embeddings",
	param=search_params,
	limit=4,
	expr=None,
	# set the names of the fields you want to retrieve from the search result.
	output_fields=['words','person_name'],
	consistency_level="Strong"
)

In [ ]:
for i in range(0,len(results[0])):
    name = results[0][i].entity.get('words')
    pname = results[0][i].entity.get('person_name')
    print(pname)
#     try:
#         display(Image.open('./images/'+name+'.jpg'))
#     except:
#         display(Image.open('./images/'+name+'.png'))

Auranzaib
Auranzaib
Qasim
Hasnant


In [ ]:
labels

['a2', 'a1', 'h1', 'q2', 'h2', 'q1']

# Langchain Function Calling

In [ ]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY_1')

In [ ]:
%pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': 'bad6d5d6-8a2e-4135-b207-921a2e6acf77', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '4119be62-424a-42c0-98b0-f898ecb64c6e', 'type': 'tool_call'}]


In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

print(messages)




[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-74fdfceb-8bdb-4e57-af0e-4cf4982ca236-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': 'bad6d5d6-8a2e-4135-b207-921a2e6acf77', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '4119be62-424a-42c0-98b0-f898ecb64c6e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 115, 'output_tokens': 6, 'total_tokens': 121, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='36', name='multiply', tool_call_id='bad6d5d6-8a2e-4135-b207-921a2e6acf77'), ToolMessage(content='60', name='add', tool_call_id='4119be62-424a-42c0-98b0-f898ecb64c6e'), ToolMessage(content='36', name='multip

In [ ]:
print(llm_with_tools.invoke(messages).content)

3 * 12 is 36. 11 + 49 is 60.


# Langchain tool calling

In [ ]:
!pip install --upgrade stackapi

In [ ]:
from langchain_community.utilities import StackExchangeAPIWrapper

@tool
def stackexchange(query: str) -> str:
  """Executes a search query on StackExchange and returns relevant answers or posts.
  - `query`: The user's search or troubleshooting question.
  - Returns: Top results from StackExchange."""
  return StackExchangeAPIWrapper().run(query)

In [ ]:
tools = [stackexchange]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage

query = "zsh: command not found: python"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'stackexchange', 'args': {'query': 'zsh: command not found: python'}, 'id': 'f3ddc76e-a8f7-4d2b-9137-81f384fa10c9', 'type': 'tool_call'}]


In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"stackexchange": stackexchange}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

print(messages)




[HumanMessage(content='zsh: command not found: python', additional_kwargs={}, response_metadata={}), AIMessage(content="It seems like `python` isn't recognized by your shell.  This usually means it's not installed or not in your system's PATH. Here's how to troubleshoot:\n\n1. **Check if Python is installed:**\n\n   Open a new terminal and try:\n   ```bash\n   which python\n   which python3\n   ```\n   If either command returns a path (e.g., `/usr/bin/python3`), Python is installed, but might not be in your PATH.  If neither command returns a path, Python isn't installed.\n\n2. **Add Python to your PATH (if installed but not in PATH):**\n\n   The process for adding to your PATH depends on your shell (zsh, bash, etc.) and operating system. Here are a few common examples:\n\n   * **zsh:** Add `export PATH=$PATH:/path/to/your/python` to your `.zshrc` file. Replace `/path/to/your/python` with the actual path from step 1.  Then, run `source ~/.zshrc`.\n\n   * **bash:** Add `export PATH=$PAT

In [ ]:
print(llm_with_tools.invoke(messages).content)

This error usually means that the `python` command is not found in your system's PATH.  Here's how to fix it:

1. **Make sure Python is installed:**

   - **macOS/Linux:** Open a terminal and type `which python3`. If it returns a path, Python 3 is installed. If not, or if you want Python 2, use `which python`. If neither are found, you need to install Python.  You can download it from [python.org](https://www.python.org/downloads/).
   - **Windows:** Open a command prompt and type `where python`. If it returns a path, Python is installed. If not, you need to install it. You can download it from [python.org](https://www.python.org/downloads/).

2. **Add Python to your PATH:** If Python is installed but not in your PATH, you need to add it.

   - **macOS/Linux:**  You'll need to edit your shell's configuration file (e.g., `.bashrc`, `.zshrc`, `.bash_profile`). Add a line like `export PATH="$PATH:/path/to/python"` (replace `/path/to/python` with the actual path to your Python installation